# Usando os truques de treinamento de redes neurais!

A documentação do Keras está em https://www.tensorflow.org/api_docs/python/tf/keras/

In [1]:
%matplotlib inline
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

np.random.seed(42)

mpl.rc("axes", labelsize=14)
mpl.rc("xtick", labelsize=12)
mpl.rc("ytick", labelsize=12)

In [2]:
# GAMBIARRA: https://stackoverflow.com/questions/69687794/unable-to-manually-load-cifar10-dataset
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
import tensorflow as tf
from tensorflow import keras

tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [4]:
# Lê o dataset CIFAR-10, https://keras.io/api/datasets/cifar10/
train_data, test_data = keras.datasets.cifar10.load_data()
X_train_full, y_train_full = train_data
X_test, y_test = test_data

170498071/170498071 [==============================] - 11s 0us/step


Fonte: https://www.cs.toronto.edu/~kriz/cifar.html

<img src='cifar.png'></img>

O conjunto de treinamento tem $50000$ imagens de $32 \times 32$ pixels, com $3$ canais de cor. Portanto, o *array* `X_train_full` contendo esses dados tem formato $(\text{amostras}, \text{linhas}, \text{colunas}, \text{canais}) = (50000, 32, 32, 3)$

In [5]:
print(X_train_full.shape)

(50000, 32, 32, 3)


Cada canal de cor é representado como um inteiro entre $0$ e $255$, e isso cabe em um único *byte*. Para economizar espaço, o tipo de dados do *array* então será `uint8`, significando "inteiro sem-sinal de 8 bits". Ou para o povo do C/C++, o famoso `unsigned char`.

In [6]:
print(X_train_full.dtype)

uint8


A variável dependente `y_train_full` tem $50000$ amostras e apenas um valor por amostra. Para se manter consistente com os formatos de dados do *framework* Tensorflow, a variável dependente será representada por uma matriz-coluna de tamanho $(50000, 1)$ ao invés de um array simples de tamanho $(50000,)$

In [7]:
print(y_train_full.shape)

(50000, 1)


Como as classes são inteiros entre $0$ e $9$, um `uint8` basta para armazená-las

In [8]:
print(y_train_full.dtype)

uint8


Já o conjunto de testes tem $10000$ amostras, observe os tamanhos e tipos de dado de `X_test` e `y_test`:

In [9]:
print(X_test.shape, X_test.dtype)
print(y_test.shape, y_test.dtype)

(10000, 32, 32, 3) uint8
(10000, 1) uint8


O conjunto de teste, como de costume, é inviolável. Vamos dividir o conjunto de treinamento mais uma vez em duas partes: um conjunto que realmente será usado para treinamento, e outro para validação (que é o conjunto de testes de dentro do conjunto de treinamento pleno, e então não é inviolável).

Vamos separar $5000$ amostras para validação.

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full,
    y_train_full,
    test_size=5000,
)

print("X_train", X_train.shape, X_train.dtype)
print("y_train", y_train.shape, y_train.dtype)
print("X_valid", X_valid.shape, X_valid.dtype)
print("y_valid", y_valid.shape, y_valid.dtype)

X_train (45000, 32, 32, 3) uint8
y_train (45000, 1) uint8
X_valid (5000, 32, 32, 3) uint8
y_valid (5000, 1) uint8


Antes de mais nada, vamos criar *baselines* de desempenho para comparação: o classificador trivial e a Random Forest.

O classificador mais simples, mais trivial possível, é simplesmente dizer que toda amostra vem da mesma classe: a classe com maior representação no conjunto de treinamento. Vamos ver:

In [11]:
import pandas as pd

pd.Series(y_train.ravel()).value_counts(True).sort_values()

2    0.099400
3    0.099489
7    0.099800
5    0.099911
8    0.099933
9    0.100000
1    0.100289
4    0.100311
6    0.100333
0    0.100533
dtype: float64

Aparentemente todas as classes estão representadas de modo aproximadamente igual, sendo a maior proporção igual a $10.05 \%$. Então vamos adotar como classe "oficial do nosso classificador trivial a classe $0$. Agora vamos ver o desempenho no conjunto de validação:

In [12]:
pd.Series(y_valid.ravel()).value_counts(True).sort_values()

0    0.0952
6    0.0970
4    0.0972
1    0.0974
9    0.1000
8    0.1006
5    0.1008
7    0.1018
3    0.1046
2    0.1054
dtype: float64

Neste conjunto a classe $0$ tem, por acidente estatístico, a pior proporção! Paciência, a vida é assim! (Procure pelo fenômeno de "regressão para a média", muitas coisas da vida vão ficar mais claras...)

Portanto, o nosso desempenho do classificador trivial será uma acurácia de $9.5\%$.

Se você quiser uma estimativa melhor de desempenho do classificador trivial você pode fazer validação cruzada, ou mesmo estimar a estatística de acurácia usando *bootstrap* para obter uma distribuição do valor de acurácia. Mas nada disso será relevante: $9.5\%$ é muito baixo.

Vamos ver agora a RandomForest.

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [14]:
%%time
model = RandomForestClassifier(n_jobs=-1, random_state=42)
model.fit(X_train.reshape(X_train.shape[0], -1), y_train.ravel())

CPU times: total: 7min 43s
Wall time: 36.6 s


RandomForestClassifier(n_jobs=-1, random_state=42)

In [15]:
y_pred = model.predict(X_valid.reshape(X_valid.shape[0], -1))
print(accuracy_score(y_valid.ravel(), y_pred))

0.454


Ok, esse é o *score* a ser vencido! (Será? Você pode pensar em alguma ressalva aqui?)

Vamos agora construir uma rede neural das antigas: nada profunda, e com função de ativação sigmoide. Vamos usar um `batch_size` grande, pois os itens do dataset são pequenos, não precisamos de batches muito pequenos (o tamanho padrão é 32 no TensorFlow). Veremos o desempenho:

In [16]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
model.add(keras.layers.Dense(100, activation="sigmoid"))
model.add(keras.layers.Dense(10, activation="softmax"))

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="sgd",
    metrics=["accuracy"],
)

run_logdir = os.path.join(os.curdir, "cifar10_logs", "inicial")
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [tensorboard_cb]

model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=1024,
    validation_data=(X_valid, y_valid),
    callbacks=callbacks,
)
model.evaluate(X_valid, y_valid)

Epoch 1/100
44/44 [==============================] - 1s 19ms/step - loss: 2.2695 - accuracy: 0.1617 - val_loss: 2.2007 - val_accuracy: 0.1960
Epoch 2/100
44/44 [==============================] - 1s 14ms/step - loss: 2.1742 - accuracy: 0.2118 - val_loss: 2.1798 - val_accuracy: 0.1970
Epoch 3/100
44/44 [==============================] - 1s 13ms/step - loss: 2.1430 - accuracy: 0.2335 - val_loss: 2.1354 - val_accuracy: 0.2360
Epoch 4/100
44/44 [==============================] - 1s 14ms/step - loss: 2.1062 - accuracy: 0.2513 - val_loss: 2.1050 - val_accuracy: 0.2426
Epoch 5/100
44/44 [==============================] - 1s 13ms/step - loss: 2.0804 - accuracy: 0.2631 - val_loss: 2.0796 - val_accuracy: 0.2698
Epoch 6/100
44/44 [==============================] - 1s 13ms/step - loss: 2.0548 - accuracy: 0.2730 - val_loss: 2.0739 - val_accuracy: 0.2606
Epoch 7/100
44/44 [==============================] - 1s 13ms/step - loss: 2.0332 - accuracy: 0.2810 - val_loss: 2.0373 - val_accuracy: 0.2602
Epoch 

[1.8087332248687744, 0.35040000081062317]

Vamos partir para uma rede profunda agora. Vamos criar $10$ camadas *fully-connected*, função de ativação "elu", inicialização de pesos adequada para a respectiva função de ativação, e otimizador "adam". Vamos ver se melhora:

In [17]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(10):
    model.add(
        keras.layers.Dense(
            100,
            kernel_initializer="he_normal",
            activation="elu",
        ))
model.add(
    keras.layers.Dense(
        10,
        kernel_initializer="glorot_normal",
        activation="softmax",
    ))

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)

run_logdir = os.path.join(os.curdir, "cifar10_logs", "profunda")
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [tensorboard_cb]

model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=1024,
    validation_data=(X_valid, y_valid),
    callbacks=callbacks,
)
model.evaluate(X_valid, y_valid)

Epoch 1/100
44/44 [==============================] - 3s 29ms/step - loss: 92.2688 - accuracy: 0.1092 - val_loss: 21.6370 - val_accuracy: 0.1124
Epoch 2/100
44/44 [==============================] - 1s 23ms/step - loss: 10.6197 - accuracy: 0.1503 - val_loss: 3.5494 - val_accuracy: 0.1542
Epoch 3/100
44/44 [==============================] - 1s 23ms/step - loss: 3.3607 - accuracy: 0.1699 - val_loss: 5.0637 - val_accuracy: 0.1446
Epoch 4/100
44/44 [==============================] - 1s 24ms/step - loss: 4.2721 - accuracy: 0.1583 - val_loss: 2.8760 - val_accuracy: 0.1662
Epoch 5/100
44/44 [==============================] - 1s 24ms/step - loss: 2.4503 - accuracy: 0.2070 - val_loss: 2.2729 - val_accuracy: 0.2218
Epoch 6/100
44/44 [==============================] - 1s 24ms/step - loss: 2.2229 - accuracy: 0.2311 - val_loss: 2.1842 - val_accuracy: 0.2334
Epoch 7/100
44/44 [==============================] - 1s 24ms/step - loss: 2.1852 - accuracy: 0.2355 - val_loss: 2.1433 - val_accuracy: 0.2364
Epo

[1.6749637126922607, 0.41440001130104065]

Muito melhor! Agora acertamos em torno de $43\%$ das classes!

A técnica de "batch normalization" pode melhorar ainda mais a convergência do modelo, vamos experimentar:

In [18]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(10):
    model.add(keras.layers.BatchNormalization())
    model.add(
        keras.layers.Dense(
            100,
            kernel_initializer="he_normal",
            activation="elu",
            use_bias=False,
        ))
model.add(
    keras.layers.Dense(
        10,
        kernel_initializer="glorot_normal",
        activation="softmax",
    ))

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)

run_logdir = os.path.join(os.curdir, "cifar10_logs", "batchnorm")
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [tensorboard_cb]

model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=1024,
    validation_data=(X_valid, y_valid),
    callbacks=callbacks,
)
model.evaluate(X_valid, y_valid)

Epoch 1/100
44/44 [==============================] - 6s 45ms/step - loss: 1.8820 - accuracy: 0.3428 - val_loss: 4.2016 - val_accuracy: 0.1514
Epoch 2/100
44/44 [==============================] - 2s 38ms/step - loss: 1.5595 - accuracy: 0.4437 - val_loss: 2.2291 - val_accuracy: 0.3022
Epoch 3/100
44/44 [==============================] - 2s 39ms/step - loss: 1.4395 - accuracy: 0.4884 - val_loss: 1.7787 - val_accuracy: 0.4076
Epoch 4/100
44/44 [==============================] - 2s 38ms/step - loss: 1.3532 - accuracy: 0.5184 - val_loss: 1.6168 - val_accuracy: 0.4508
Epoch 5/100
44/44 [==============================] - 2s 38ms/step - loss: 1.2807 - accuracy: 0.5449 - val_loss: 1.5643 - val_accuracy: 0.4594
Epoch 6/100
44/44 [==============================] - 2s 39ms/step - loss: 1.2224 - accuracy: 0.5665 - val_loss: 1.5036 - val_accuracy: 0.4758
Epoch 7/100
44/44 [==============================] - 2s 46ms/step - loss: 1.1725 - accuracy: 0.5847 - val_loss: 1.5268 - val_accuracy: 0.4730
Epoch 

[4.94550085067749, 0.448199987411499]

Uau. Isso sim que é *overfitting*! Para resolver o problema do *overfitting* vamos adotar o EarlyStopping:

In [19]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
run_logdir = os.path.join(os.curdir, "cifar10_logs", "earlystopping")

# Criação do modelo.
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(10):
    model.add(keras.layers.BatchNormalization())
    model.add(
        keras.layers.Dense(
            100,
            kernel_initializer="he_normal",
            activation="elu",
            use_bias=False,
        ))
model.add(
    keras.layers.Dense(
        10,
        kernel_initializer="glorot_normal",
        activation="softmax",
    ))

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)

# Otimização do modelo.
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10)
callbacks = [tensorboard_cb, early_stopping_cb]

model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=1024,
    validation_data=(X_valid, y_valid),
    callbacks=callbacks,
)

# Avaliação do modelo.
model.evaluate(X_valid, y_valid)

Epoch 1/100
44/44 [==============================] - 6s 52ms/step - loss: 1.8875 - accuracy: 0.3363 - val_loss: 6.0531 - val_accuracy: 0.1192
Epoch 2/100
44/44 [==============================] - 2s 44ms/step - loss: 1.5593 - accuracy: 0.4426 - val_loss: 2.7272 - val_accuracy: 0.2768
Epoch 3/100
44/44 [==============================] - 2s 42ms/step - loss: 1.4440 - accuracy: 0.4859 - val_loss: 1.8436 - val_accuracy: 0.3838
Epoch 4/100
44/44 [==============================] - 2s 42ms/step - loss: 1.3566 - accuracy: 0.5186 - val_loss: 1.6524 - val_accuracy: 0.4324
Epoch 5/100
44/44 [==============================] - 2s 42ms/step - loss: 1.2844 - accuracy: 0.5426 - val_loss: 1.5320 - val_accuracy: 0.4656
Epoch 6/100
44/44 [==============================] - 2s 43ms/step - loss: 1.2231 - accuracy: 0.5666 - val_loss: 1.5007 - val_accuracy: 0.4704
Epoch 7/100
44/44 [==============================] - 2s 44ms/step - loss: 1.1692 - accuracy: 0.5874 - val_loss: 1.4949 - val_accuracy: 0.4788
Epoch 

[1.744353175163269, 0.48019999265670776]

Melhorou mais! Mas ainda temos o problema do *overfitting*, vamos tentar a técnica do "drop-out" para ver se melhora:

In [20]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
run_logdir = os.path.join(os.curdir, "cifar10_logs", "dropout")

# Criação do modelo.
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(10):
    model.add(keras.layers.Dropout(rate=0.1))
    model.add(keras.layers.BatchNormalization())
    model.add(
        keras.layers.Dense(
            100,
            kernel_initializer="he_normal",
            activation="elu",
            use_bias=False,
        ))
model.add(
    keras.layers.Dense(
        10,
        kernel_initializer="glorot_normal",
        activation="softmax",
    ))

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)

# Otimização do modelo.
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10)
callbacks = [tensorboard_cb, early_stopping_cb]

model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=1024,
    validation_data=(X_valid, y_valid),
    callbacks=callbacks,
)

# Avaliação do modelo.
model.evaluate(X_valid, y_valid)

Epoch 1/100
44/44 [==============================] - 7s 66ms/step - loss: 2.1554 - accuracy: 0.2460 - val_loss: 4.5805 - val_accuracy: 0.1732
Epoch 2/100
44/44 [==============================] - 3s 58ms/step - loss: 1.8301 - accuracy: 0.3385 - val_loss: 2.4312 - val_accuracy: 0.2976
Epoch 3/100
44/44 [==============================] - 3s 61ms/step - loss: 1.7413 - accuracy: 0.3717 - val_loss: 1.9805 - val_accuracy: 0.3692
Epoch 4/100
44/44 [==============================] - 3s 60ms/step - loss: 1.6791 - accuracy: 0.3950 - val_loss: 1.7074 - val_accuracy: 0.4148
Epoch 5/100
44/44 [==============================] - 3s 61ms/step - loss: 1.6348 - accuracy: 0.4167 - val_loss: 1.6313 - val_accuracy: 0.4320
Epoch 6/100
44/44 [==============================] - 3s 61ms/step - loss: 1.5964 - accuracy: 0.4304 - val_loss: 1.5596 - val_accuracy: 0.4532
Epoch 7/100
44/44 [==============================] - 3s 62ms/step - loss: 1.5639 - accuracy: 0.4424 - val_loss: 1.5313 - val_accuracy: 0.4620
Epoch 

[1.330750584602356, 0.5347999930381775]

Melhorou ainda mais! E agora não tem mais tanto overfitting! Já que é assim, vamos aumentar ainda mais a rede e ver se melhora: vamos colocar $20$ camadas!

In [21]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
run_logdir = os.path.join(os.curdir, "cifar10_logs", "mais_profunda")

# Criação do modelo.
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dropout(rate=0.1))
    model.add(keras.layers.BatchNormalization())
    model.add(
        keras.layers.Dense(
            100,
            kernel_initializer="he_normal",
            activation="elu",
            use_bias=False,
        ))
model.add(
    keras.layers.Dense(
        10,
        kernel_initializer="glorot_normal",
        activation="softmax",
    ))

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="nadam",
    metrics=["accuracy"],
)

# Otimização do modelo.
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10)
callbacks = [tensorboard_cb, early_stopping_cb]

model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=1024,
    validation_data=(X_valid, y_valid),
    callbacks=callbacks,
)

# Avaliação do modelo.
model.evaluate(X_valid, y_valid)

Epoch 1/100
44/44 [==============================] - 18s 105ms/step - loss: 2.4825 - accuracy: 0.1364 - val_loss: 4.4491 - val_accuracy: 0.1012
Epoch 2/100
44/44 [==============================] - 4s 94ms/step - loss: 2.0872 - accuracy: 0.2200 - val_loss: 4.8384 - val_accuracy: 0.1556
Epoch 3/100
44/44 [==============================] - 4s 91ms/step - loss: 1.9505 - accuracy: 0.2729 - val_loss: 3.0733 - val_accuracy: 0.2390
Epoch 4/100
44/44 [==============================] - 4s 91ms/step - loss: 1.8829 - accuracy: 0.3061 - val_loss: 2.3973 - val_accuracy: 0.3088
Epoch 5/100
44/44 [==============================] - 4s 93ms/step - loss: 1.8295 - accuracy: 0.3311 - val_loss: 2.0593 - val_accuracy: 0.3496
Epoch 6/100
44/44 [==============================] - 4s 96ms/step - loss: 1.7797 - accuracy: 0.3535 - val_loss: 1.8728 - val_accuracy: 0.3822
Epoch 7/100
44/44 [==============================] - 4s 92ms/step - loss: 1.7369 - accuracy: 0.3707 - val_loss: 1.7776 - val_accuracy: 0.4012
Epoc

[1.3625441789627075, 0.5342000126838684]

Parece que não melhorou muito, estava bom do jeito anterior. Para terminar, vamos usar um escalonador de taxa de aprendizado para melhorar a velocidade de convergência do nosso modelo.

In [22]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
run_logdir = os.path.join(os.curdir, "cifar10_logs", "escalonador")

# Criação do modelo.
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(10):
    model.add(keras.layers.Dropout(rate=0.1))
    model.add(keras.layers.BatchNormalization())
    model.add(
        keras.layers.Dense(
            100,
            kernel_initializer="he_normal",
            activation="elu",
            use_bias=False,
        ))
model.add(
    keras.layers.Dense(
        10,
        kernel_initializer="glorot_normal",
        activation="softmax",
    ))

s = 90 * len(X_train) // 1024  # number of steps in 90 epochs (batch size 1024)
learning_rate = keras.optimizers.schedules.ExponentialDecay(0.01, s, 0.1)
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=optimizer,
    metrics=["accuracy"],
)

# Otimização do modelo.
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10)
callbacks = [tensorboard_cb, early_stopping_cb]

model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=1024,
    validation_data=(X_valid, y_valid),
    callbacks=callbacks,
)

# Avaliação do modelo.
model.evaluate(X_valid, y_valid)

Epoch 1/100
44/44 [==============================] - 7s 68ms/step - loss: 1.9836 - accuracy: 0.2921 - val_loss: 41.1905 - val_accuracy: 0.1622
Epoch 2/100
44/44 [==============================] - 3s 61ms/step - loss: 1.7398 - accuracy: 0.3749 - val_loss: 4.7053 - val_accuracy: 0.3120
Epoch 3/100
44/44 [==============================] - 3s 62ms/step - loss: 1.6637 - accuracy: 0.4042 - val_loss: 2.6776 - val_accuracy: 0.3518
Epoch 4/100
44/44 [==============================] - 3s 60ms/step - loss: 1.5965 - accuracy: 0.4279 - val_loss: 1.9223 - val_accuracy: 0.4076
Epoch 5/100
44/44 [==============================] - 3s 61ms/step - loss: 1.5524 - accuracy: 0.4444 - val_loss: 1.6819 - val_accuracy: 0.4352
Epoch 6/100
44/44 [==============================] - 3s 60ms/step - loss: 1.5147 - accuracy: 0.4605 - val_loss: 1.6015 - val_accuracy: 0.4610
Epoch 7/100
44/44 [==============================] - 3s 61ms/step - loss: 1.4773 - accuracy: 0.4742 - val_loss: 1.5131 - val_accuracy: 0.4686
Epoch

[1.3475792407989502, 0.5281999707221985]

In [23]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 1.3103 - accuracy: 0.5467


[1.3102915287017822, 0.5467000007629395]

**Atividade**: Refaça a atividade da aula anterior (Fashion MNIST) mas tente usar os truques de melhoramento de desempenho das redes neurais vistos aqui nesta aula.